In [ ]:
# Enhanced Code Golf Solution Analyzer with Multiple Visualizations
# Install required packages
import subprocess
import sys

print("🔧 Installing required packages...")
subprocess.check_call([sys.executable, "-m", "pip", "install", 
                      "python-minifier", "matplotlib", "seaborn", "tqdm", 
                      "numpy", "pandas", "scipy", "plotly", "scikit-learn", 
                      "wordcloud", "networkx"])

import os
from python_minifier import minify
import warnings
from zlib import compress
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
import time
import tempfile
import shutil
from scipy import stats as scipy_stats
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
from wordcloud import WordCloud
import networkx as nx
from datetime import datetime
import re

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings("ignore")

# ASCII Art Header
print("\n" + "="*100)
print("""
╔═══════════════════════════════════════════════════════════════════════════════════════════════╗
║     🔬 ULTRA-ENHANCED CODE GOLF SOLUTION ANALYZER & SUBMISSION OPTIMIZER 🔬                  ║
║                                                                                               ║
║     Advanced Analytics • Multi-Source Comparison • Performance Insights                       ║
║     Final Submission: golf-code-optimization-trials                                          ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
""")
print("="*100)

# The PROVEN submission to use for final output - Using optimization trials
PROVEN_SUBMISSION = "/kaggle/input/golf-code-optimization-trials/submission.zip"

# Source directories for ANALYSIS
analysis_sources = [
    "/kaggle/input/arc-code-golf-starter-solutions",
    # Author: https://www.kaggle.com/cristianocalcagno
    "/kaggle/input/google-code-golf-2025-submit",
    # Author: https://www.kaggle.com/tonylica
    "/kaggle/input/source-code-for-full-400-solutions",
    # Full 400 solutions
    "/kaggle/input/400-task-with-smart-solution-search-verification/source/big-zippa",
    # Alternative solution for comparison
    "/kaggle/input/neuroips-4-of-some-lessons-learned",
    # Neuroips-4 lessons learned
    "/kaggle/input/solutions-for-all-400-tasks-ensemble",
    # Ensemble solution for comparison
    "/kaggle/input/golf-code-optimization-trials",
    # Optimization trials - USED FOR SUBMISSION
]

# Known failing solutions to skip in analysis
failing = [
    "/kaggle/input/google-code-golf-2025-submit/task071.py",
    "/kaggle/input/google-code-golf-2025-submit/task074.py", 
    "/kaggle/input/google-code-golf-2025-submit/task343.py",
]

print(f"\n📁 ANALYSIS CONFIGURATION:")
print(f"   📊 Sources for analysis: {len(analysis_sources)}")
for i, source in enumerate(analysis_sources, 1):
    source_name = os.path.basename(source)
    if "golf-code-optimization-trials" in source:
        print(f"   {i}. 🌟 {source_name} [SUBMISSION SOURCE]")
    else:
        print(f"   {i}. 📂 {source_name}")

print(f"\n✅ Final submission will use: {os.path.basename(PROVEN_SUBMISSION)}")
print(f"❌ Failing solutions to skip: {len(failing)} files")
print(f"📅 Analysis timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Function to extract submission.zip if needed
def extract_submission_if_needed(source_path, for_analysis=True):
    """
    Check if submission.zip exists in the source path and extract it if needed
    Returns the path to use for loading .py files
    """
    submission_zip = os.path.join(source_path, "submission.zip")
    
    # First check if .py files exist directly
    if os.path.exists(source_path):
        has_py_files = any(f.endswith('.py') for f in os.listdir(source_path) if os.path.isfile(os.path.join(source_path, f)))
        
        if has_py_files:
            if for_analysis:
                print(f"   ✅ Found .py files directly in {os.path.basename(source_path)}")
            return source_path
        elif os.path.exists(submission_zip):
            if for_analysis:
                print(f"   📦 Found submission.zip in {os.path.basename(source_path)}, extracting for analysis...")
            # Create a temporary directory for extraction
            temp_dir = tempfile.mkdtemp()
            with zipfile.ZipFile(submission_zip, 'r') as zipf:
                zipf.extractall(temp_dir)
            if for_analysis:
                print(f"   📤 Extracted to temporary directory")
            return temp_dir
        else:
            if for_analysis:
                print(f"   ⚠️ No .py files or submission.zip found in {os.path.basename(source_path)}")
            return None
    else:
        if for_analysis:
            print(f"   ⚠️ Source not found: {source_path}")
        return None

# Process sources for ANALYSIS
processed_sources = []
temp_dirs = []  # Keep track of temp directories to clean up later

print("\n🔍 PROCESSING SOURCE DIRECTORIES FOR ANALYSIS...")
print("-" * 60)
for source in analysis_sources:
    result = extract_submission_if_needed(source)
    if result:
        processed_sources.append(result)
        if result != source:  # It's a temp directory
            temp_dirs.append(result)

analysis_sources = processed_sources
print(f"\n✅ Successfully processed {len(analysis_sources)} sources for analysis")

def calculate_zlib_compression(src):
    """Calculate what zlib compression would achieve (for analysis only)"""
    compression_level = 9
    
    # Avoid compressed data ending with quote character
    while (compressed := compress(src, compression_level))[-1] == ord('"'): 
        src += b"#"
    
    def sanitize(b_in):
        b_out = bytearray()
        for b in b_in:
            if   b==0:         b_out += b"\\x00"
            elif b==ord("\r"): b_out += b"\\r"
            elif b==ord("\\"): b_out += b"\\\\"
            else: b_out.append(b)
        return bytes(b_out)
    
    compressed = sanitize(compressed)
    delim = b'"""' if ord("\n") in compressed or ord('"') in compressed else b'"'
    
    return b"#coding:L1\nimport zlib\nexec(zlib.decompress(bytes(" + \
           delim + compressed + delim + \
           b',"L1")))'

def analyze_code_complexity(code):
    """Analyze code complexity metrics"""
    lines = code.split('\n')
    metrics = {
        'line_count': len(lines),
        'char_count': len(code),
        'import_count': sum(1 for line in lines if line.strip().startswith('import') or line.strip().startswith('from')),
        'function_count': sum(1 for line in lines if line.strip().startswith('def ')),
        'class_count': sum(1 for line in lines if line.strip().startswith('class ')),
        'loop_count': sum(1 for line in lines if 'for ' in line or 'while ' in line),
        'comprehension_count': code.count('[') + code.count('{') - code.count('{}') - code.count('[]'),
        'lambda_count': code.count('lambda'),
        'avg_line_length': np.mean([len(line) for line in lines]) if lines else 0,
        'max_line_length': max([len(line) for line in lines]) if lines else 0,
        'empty_lines': sum(1 for line in lines if not line.strip()),
        'comment_lines': sum(1 for line in lines if line.strip().startswith('#')),
        'nested_loops': sum(1 for i, line in enumerate(lines) if 'for ' in line and i > 0 and 
                           any('for ' in lines[j] or 'while ' in lines[j] for j in range(max(0, i-5), i))),
        'unique_vars': len(set(re.findall(r'\b[a-zA-Z_]\w*\b', code))),
        'operators': sum(code.count(op) for op in ['+', '-', '*', '/', '%', '**', '//', '==', '!=', '<', '>', '<=', '>=']),
    }
    return metrics

def analyze_code_patterns(code):
    """Analyze common code patterns and techniques"""
    patterns = {
        'list_comp': len(re.findall(r'\[.*for.*in.*\]', code)),
        'dict_comp': len(re.findall(r'\{.*for.*in.*\}', code)),
        'generator': code.count('yield'),
        'decorators': code.count('@'),
        'f_strings': len(re.findall(r'f["\']', code)),
        'walrus': code.count(':='),
        'ternary': len(re.findall(r'.*if.*else.*', code)),
        'map_usage': code.count('map('),
        'filter_usage': code.count('filter('),
        'reduce_usage': code.count('reduce('),
        'lambda_usage': code.count('lambda'),
        'enumerate_usage': code.count('enumerate('),
        'zip_usage': code.count('zip('),
        'range_usage': code.count('range('),
        'slice_usage': len(re.findall(r'\[.*:.*\]', code)),
    }
    return patterns

# Initialize enhanced tracking variables
stats = {
    'task_nums': [],
    'original_sizes': [],
    'minified_sizes': [],
    'compressed_sizes': [],
    'final_sizes': [],
    'improvements': [],
    'compression_ratios': [],
    'potential_scores': [],
    'source_used': [],
    'compression_beneficial': [],
    'all_source_sizes': [],
    'complexity_metrics': [],
    'minification_reduction': [],
    'task_difficulty': [],
    'best_combination_potential': [],
    'source_availability': [],
    'code_patterns': [],
    'efficiency_score': [],
    'optimization_potential': [],
}

# Track additional metrics
source_wins = defaultdict(int)
source_task_coverage = defaultdict(set)
task_source_comparison = defaultdict(dict)
task_complexity_data = defaultdict(dict)
source_pair_wins = defaultdict(int)
task_patterns = defaultdict(list)
pattern_frequency = defaultdict(int)
optimization_techniques = defaultdict(list)
source_performance_timeline = defaultdict(list)
task_size_progression = []

# ANALYSIS PHASE - Analyze all sources
print("\n" + "="*100)
print("🔬 COMPREHENSIVE ANALYSIS PHASE")
print("="*100)
print("\nAnalyzing all available solutions with ultra-enhanced metrics...")
print("This analysis will provide deep insights into:")
print("  • Code complexity and efficiency")
print("  • Compression opportunities")
print("  • Source performance comparison")
print("  • Optimization patterns and techniques")
print("  • Task difficulty classification")

start_time = time.time()
tasks_analyzed = 0
total_code_analyzed = 0

for task_num in tqdm(range(1, 401), desc="🔍 Analyzing tasks", unit="task", ncols=100):
    
    # Analyze all sources for this task
    best_src = b"#" * 1_000_000
    original_size = 0
    best_source = "none"
    task_sizes_by_source = {}
    available_sources = []
    task_code_patterns = {}
    
    for source in analysis_sources:
        path_in = f"{source}/task{task_num:03d}.py"
        if not os.path.exists(path_in): continue
        if path_in in failing: continue
        
        try:
            with open(path_in, "r") as task_in:
                new_src = task_in.read()
                current_original_size = len(new_src.encode())
                total_code_analyzed += current_original_size
                minified = minify(new_src).encode()
                
                # Analyze complexity and patterns
                complexity = analyze_code_complexity(new_src)
                patterns = analyze_code_patterns(new_src)
                
                # Choose shortest version: original vs minified
                current_best = min([new_src.encode(), minified], key=len)
                
                # Determine source name
                if "arc-code-golf-starter" in source:
                    source_name = "arc-starter"
                elif "google-code-golf-2025" in source:
                    source_name = "google-submit"
                elif "source-code-for-full-400" in source:
                    source_name = "full-400"
                elif "big-zippa" in source:
                    source_name = "big-zippa"
                elif "neuroips-4-of-some-lessons-learned" in source:
                    source_name = "neuroips-4"
                elif "ensemble" in source:
                    source_name = "ensemble"
                elif "golf-code-optimization-trials" in source:
                    source_name = "optimization-trials"
                else:
                    source_name = os.path.basename(source)
                
                # Track coverage and metrics
                source_task_coverage[source_name].add(task_num)
                task_sizes_by_source[source_name] = len(current_best)
                task_source_comparison[task_num][source_name] = len(current_best)
                task_complexity_data[task_num][source_name] = complexity
                task_code_patterns[source_name] = patterns
                available_sources.append(source_name)
                
                # Track pattern frequency
                for pattern, count in patterns.items():
                    if count > 0:
                        pattern_frequency[pattern] += count
                
                # Check if this is the best so far
                if len(current_best) < len(best_src):
                    best_src = current_best
                    original_size = current_original_size
                    best_source = source_name
                    best_complexity = complexity
                    best_patterns = patterns
                    
        except Exception as e:
            continue
    
    # Skip if no valid source found
    if len(best_src) == 1_000_000:
        continue
    
    tasks_analyzed += 1
    task_size_progression.append((task_num, len(best_src)))
    
    # Track winner
    if best_source != "none":
        source_wins[best_source] += 1
        source_performance_timeline[best_source].append(task_num)
        
    # Analyze compression potential
    compressed_src = calculate_zlib_compression(best_src)
    improvement = len(best_src) - len(compressed_src)
    compression_beneficial = improvement > 0
    
    # Determine what would be used
    final_src = compressed_src if compression_beneficial else best_src
    
    # Calculate potential score
    potential_score = max(1, 2500 - len(final_src))
    
    # Calculate minification reduction
    minification_reduction = (original_size - len(best_src)) / original_size * 100 if original_size > 0 else 0
    
    # Calculate efficiency score (custom metric)
    efficiency_score = potential_score / (best_complexity.get('line_count', 1) + 1)
    
    # Calculate optimization potential
    if len(task_sizes_by_source) > 1:
        size_variance = np.var(list(task_sizes_by_source.values()))
        optimization_potential = size_variance / np.mean(list(task_sizes_by_source.values())) if task_sizes_by_source else 0
    else:
        optimization_potential = 0
    
    # Estimate task difficulty based on multiple factors
    if len(final_src) < 100 and best_complexity.get('line_count', 0) < 5:
        difficulty = "Easy"
    elif len(final_src) < 300 and best_complexity.get('line_count', 0) < 15:
        difficulty = "Medium"
    elif len(final_src) < 600 and best_complexity.get('line_count', 0) < 30:
        difficulty = "Hard"
    else:
        difficulty = "Expert"
    
    # Calculate best combination potential (theoretical best from all sources)
    if len(task_sizes_by_source) > 1:
        best_combination = min(task_sizes_by_source.values())
        combination_potential = len(final_src) - best_combination
    else:
        combination_potential = 0
    
    # Record optimization techniques used
    if 'best_patterns' in locals():
        for pattern, count in best_patterns.items():
            if count > 0:
                optimization_techniques[pattern].append(task_num)
    
    # Record statistics
    stats['task_nums'].append(task_num)
    stats['original_sizes'].append(original_size)
    stats['minified_sizes'].append(len(best_src))
    stats['compressed_sizes'].append(len(compressed_src))
    stats['final_sizes'].append(len(final_src))
    stats['improvements'].append(max(0, improvement))
    stats['compression_ratios'].append(len(compressed_src) / len(best_src) if len(best_src) > 0 else 1.0)
    stats['potential_scores'].append(potential_score)
    stats['source_used'].append(best_source)
    stats['compression_beneficial'].append(compression_beneficial)
    stats['all_source_sizes'].append(task_sizes_by_source)
    stats['complexity_metrics'].append(best_complexity if 'best_complexity' in locals() else {})
    stats['minification_reduction'].append(minification_reduction)
    stats['task_difficulty'].append(difficulty)
    stats['best_combination_potential'].append(combination_potential)
    stats['source_availability'].append(len(available_sources))
    stats['code_patterns'].append(task_code_patterns)
    stats['efficiency_score'].append(efficiency_score)
    stats['optimization_potential'].append(optimization_potential)
    
    # Track patterns
    task_patterns[task_num // 50].append(len(final_src))

analysis_time = time.time() - start_time

# Calculate source synergies
print("\n🔄 Calculating source synergies and optimization opportunities...")
for task_num, sources in task_source_comparison.items():
    if len(sources) >= 2:
        sorted_sources = sorted(sources.items(), key=lambda x: x[1])
        if len(sorted_sources) >= 2:
            source_pair_wins[f"{sorted_sources[0][0]} + {sorted_sources[1][0]}"] += 1

# Clean up temporary directories from analysis
for temp_dir in temp_dirs:
    shutil.rmtree(temp_dir)
print(f"🧹 Cleaned up {len(temp_dirs)} temporary directories")

# SUBMISSION PHASE - Copy the optimization-trials submission
print("\n" + "="*100)
print("📦 SUBMISSION PHASE: Using Golf Code Optimization Trials Solution")
print("="*100)

submission_path = "/kaggle/working/submission.zip"

if os.path.exists(PROVEN_SUBMISSION):
    print(f"✅ Found optimization trials submission: submission.zip")
    print(f"📋 Copying to: {submission_path}")
    shutil.copy2(PROVEN_SUBMISSION, submission_path)
    submission_size = os.path.getsize(submission_path)
    print(f"✅ Submission copied successfully!")
    print(f"📦 Final submission size: {submission_size:,} bytes ({submission_size/1024:.1f} KB)")
    print(f"📊 Size per task average: {submission_size/400:.1f} bytes")
else:
    print(f"❌ ERROR: Optimization trials submission not found at {PROVEN_SUBMISSION}")
    print(f"   Please check the path and try again.")
    submission_size = 0

# ENHANCED ANALYTICS PHASE
print("\n" + "="*100)
print("📊 ULTRA-COMPREHENSIVE ANALYSIS RESULTS")
print("="*100)

df = pd.DataFrame({k: v for k, v in stats.items() if k not in ['all_source_sizes', 'complexity_metrics', 'code_patterns']})
analyzed_tasks = len(df)
compression_beneficial_count = sum(stats['compression_beneficial'])
total_potential_score = sum(stats['potential_scores'])
total_bytes_saved = sum(stats['improvements'])

# Create comprehensive summary
print(f"\n🎯 EXECUTIVE SUMMARY")
print("─" * 60)
print(f"⏱️  Analysis Performance:")
print(f"    • Total time: {analysis_time:.2f} seconds")
print(f"    • Per-task average: {analysis_time/400:.3f} seconds")
print(f"    • Code analyzed: {total_code_analyzed:,} bytes ({total_code_analyzed/1024/1024:.2f} MB)")
print(f"    • Analysis speed: {total_code_analyzed/analysis_time/1024:.1f} KB/sec")

print(f"\n📋 Task Coverage:")
print(f"    • Tasks analyzed: {tasks_analyzed}/400 ({tasks_analyzed/400*100:.1f}%)")
print(f"    • Sources compared: {len(analysis_sources)}")
print(f"    • Total comparisons: {tasks_analyzed * len(analysis_sources)}")

print(f"\n🗜️  Compression Analysis:")
print(f"    • Compression beneficial: {compression_beneficial_count}/{analyzed_tasks} tasks")
print(f"    • Success rate: {compression_beneficial_count/analyzed_tasks*100:.1f}%")
print(f"    • Total bytes saved: {total_bytes_saved:,} bytes")
print(f"    • Average savings: {total_bytes_saved/compression_beneficial_count:.1f} bytes/task (when beneficial)")

print(f"\n🎯 Scoring Potential:")
print(f"    • Total potential score: {total_potential_score:,} points")
print(f"    • Average score per task: {total_potential_score/analyzed_tasks:.1f} points")
print(f"    • Maximum possible score: {2500 * analyzed_tasks:,} points")
print(f"    • Score efficiency: {total_potential_score/(2500*analyzed_tasks)*100:.1f}%")

# Advanced statistics
print(f"\n📈 ADVANCED SIZE STATISTICS:")
print("─" * 60)
print(f"Original sizes:")
print(f"    • Mean: {np.mean(stats['original_sizes']):.0f} bytes ± {np.std(stats['original_sizes']):.0f}")
print(f"    • Median: {np.median(stats['original_sizes']):.0f} bytes")
print(f"    • Mode: {scipy_stats.mode(stats['original_sizes'], keepdims=True)[0][0]:.0f} bytes")
print(f"    • Range: [{min(stats['original_sizes'])}, {max(stats['original_sizes'])}]")
print(f"    • IQR: Q1={np.percentile(stats['original_sizes'], 25):.0f}, Q3={np.percentile(stats['original_sizes'], 75):.0f}")
print(f"    • Skewness: {scipy_stats.skew(stats['original_sizes']):.2f}")
print(f"    • Kurtosis: {scipy_stats.kurtosis(stats['original_sizes']):.2f}")

print(f"\nFinal sizes:")
print(f"    • Mean: {np.mean(stats['final_sizes']):.0f} bytes ± {np.std(stats['final_sizes']):.0f}")
print(f"    • Median: {np.median(stats['final_sizes']):.0f} bytes")
print(f"    • Mode: {scipy_stats.mode(stats['final_sizes'], keepdims=True)[0][0]:.0f} bytes")
print(f"    • Range: [{min(stats['final_sizes'])}, {max(stats['final_sizes'])}]")
print(f"    • IQR: Q1={np.percentile(stats['final_sizes'], 25):.0f}, Q3={np.percentile(stats['final_sizes'], 75):.0f}")

print(f"\nSize reduction statistics:")
total_reduction = sum(stats['original_sizes']) - sum(stats['final_sizes'])
print(f"    • Total reduction: {total_reduction:,} bytes")
print(f"    • Average reduction: {total_reduction/analyzed_tasks:.1f} bytes/task")
print(f"    • Reduction percentage: {total_reduction/sum(stats['original_sizes'])*100:.1f}%")

print(f"\nMinification effectiveness:")
print(f"    • Average reduction: {np.mean(stats['minification_reduction']):.1f}%")
print(f"    • Median reduction: {np.median(stats['minification_reduction']):.1f}%")
print(f"    • Max reduction: {np.max(stats['minification_reduction']):.1f}%")
print(f"    • Tasks with >50% reduction: {sum(1 for r in stats['minification_reduction'] if r > 50)}")

# Task difficulty distribution
difficulty_counts = pd.Series(stats['task_difficulty']).value_counts()
print(f"\n🎯 TASK DIFFICULTY DISTRIBUTION:")
print("─" * 60)
for difficulty, count in difficulty_counts.items():
    difficulty_emoji = {'Easy': '🟢', 'Medium': '🟡', 'Hard': '🔴', 'Expert': '🟣'}.get(difficulty, '⚪')
    bar_length = int(count / analyzed_tasks * 50)
    bar = '█' * bar_length + '░' * (50 - bar_length)
    print(f"    {difficulty_emoji} {difficulty:8s}: {bar} {count:3d} ({count/analyzed_tasks*100:5.1f}%)")

# Source performance analysis
print(f"\n🏆 SOURCE PERFORMANCE ANALYSIS:")
print("─" * 60)
source_counts = pd.Series(stats['source_used']).value_counts()
for i, (source, count) in enumerate(source_counts.items(), 1):
    medal = {1: '🥇', 2: '🥈', 3: '🥉'}.get(i, '  ')
    bar_length = int(count / analyzed_tasks * 40)
    bar = '█' * bar_length + '░' * (40 - bar_length)
    # Special marking for optimization-trials
    if source == "optimization-trials":
        print(f"    {medal} {source:20s}: {bar} {count:3d} wins ({count/analyzed_tasks*100:5.1f}%) 🌟 SUBMISSION")
    else:
        print(f"    {medal} {source:20s}: {bar} {count:3d} wins ({count/analyzed_tasks*100:5.1f}%)")

# Source availability analysis
print(f"\n📊 SOURCE AVAILABILITY ANALYSIS:")
print("─" * 60)
avg_sources_per_task = np.mean(stats['source_availability'])
print(f"    • Average sources available per task: {avg_sources_per_task:.2f}")
print(f"    • Tasks with all {len(analysis_sources)} sources: {sum(1 for s in stats['source_availability'] if s == len(analysis_sources))}")
print(f"    • Tasks with only 1 source: {sum(1 for s in stats['source_availability'] if s == 1)}")
print(f"    • Most common availability: {scipy_stats.mode(stats['source_availability'], keepdims=True)[0][0]} sources")

# Code pattern analysis
print(f"\n🔍 CODE PATTERN ANALYSIS (Top Techniques):")
print("─" * 60)
sorted_patterns = sorted(pattern_frequency.items(), key=lambda x: x[1], reverse=True)[:10]
for pattern, frequency in sorted_patterns:
    pattern_name = pattern.replace('_', ' ').title()
    print(f"    • {pattern_name:20s}: {frequency:4d} occurrences")

# Optimization insights
print(f"\n💡 OPTIMIZATION INSIGHTS:")
print("─" * 60)
high_potential_tasks = [i for i, p in enumerate(stats['optimization_potential']) if p > np.percentile(stats['optimization_potential'], 75)]
print(f"    • Tasks with high optimization potential: {len(high_potential_tasks)}")
print(f"    • Average efficiency score: {np.mean(stats['efficiency_score']):.2f}")
print(f"    • Best efficiency score: {np.max(stats['efficiency_score']):.2f}")
if high_potential_tasks:
    task_list = ', '.join([f'Task {stats["task_nums"][i]:03d}' for i in high_potential_tasks[:5]])
    print(f"    • Top optimization candidates: {task_list}")

# =============================================================================
# MULTIPLE VISUALIZATIONS PHASE
# =============================================================================
print("\n" + "="*100)
print("📊 GENERATING MULTIPLE ENHANCED VISUALIZATIONS")
print("="*100)

# 1. SIZE DISTRIBUTION ANALYSIS
print("\n📊 Creating Size Distribution Analysis...")
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Code Golf Size Distribution Analysis', fontsize=16, fontweight='bold')

# Original size distribution
axes[0,0].hist(stats['original_sizes'], bins=50, alpha=0.7, color='#3498db', edgecolor='black')
axes[0,0].axvline(np.mean(stats['original_sizes']), color='red', linestyle='--', linewidth=2)
axes[0,0].axvline(np.median(stats['original_sizes']), color='green', linestyle='--', linewidth=2)
axes[0,0].set_xlabel('Original Size (bytes)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_title('Original Size Distribution')
axes[0,0].legend(['Mean', 'Median', 'Data'])

# Minified size distribution
axes[0,1].hist(stats['minified_sizes'], bins=50, alpha=0.7, color='#9b59b6', edgecolor='black')
axes[0,1].axvline(np.mean(stats['minified_sizes']), color='red', linestyle='--', linewidth=2)
axes[0,1].axvline(np.median(stats['minified_sizes']), color='green', linestyle='--', linewidth=2)
axes[0,1].set_xlabel('Minified Size (bytes)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Minified Size Distribution')

# Final size distribution
axes[0,2].hist(stats['final_sizes'], bins=50, alpha=0.7, color='#2ecc71', edgecolor='black')
axes[0,2].axvline(np.mean(stats['final_sizes']), color='red', linestyle='--', linewidth=2)
axes[0,2].axvline(np.median(stats['final_sizes']), color='green', linestyle='--', linewidth=2)
axes[0,2].set_xlabel('Final Size (bytes)')
axes[0,2].set_ylabel('Frequency')
axes[0,2].set_title('Final Size Distribution')

# Size reduction violin plot
reduction_data = [stats['original_sizes'], stats['minified_sizes'], stats['final_sizes']]
axes[1,0].violinplot(reduction_data, positions=[1, 2, 3], showmeans=True, showmedians=True)
axes[1,0].set_xticks([1, 2, 3])
axes[1,0].set_xticklabels(['Original', 'Minified', 'Final'])
axes[1,0].set_ylabel('Size (bytes)')
axes[1,0].set_title('Size Reduction Stages')
axes[1,0].grid(True, alpha=0.3)

# Compression ratio distribution
axes[1,1].hist(stats['compression_ratios'], bins=30, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[1,1].axvline(1.0, color='black', linestyle='--', linewidth=2, label='No compression')
axes[1,1].set_xlabel('Compression Ratio')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Compression Effectiveness')
axes[1,1].legend()

# Potential score distribution
axes[1,2].hist(stats['potential_scores'], bins=50, alpha=0.7, color='#f39c12', edgecolor='black')
axes[1,2].axvline(np.mean(stats['potential_scores']), color='red', linestyle='--', linewidth=2)
axes[1,2].set_xlabel('Potential Score')
axes[1,2].set_ylabel('Frequency')
axes[1,2].set_title('Score Distribution')

plt.tight_layout()
plt.savefig('/kaggle/working/size_distribution_analysis.png', dpi=150, bbox_inches='tight')
print("✅ Saved: size_distribution_analysis.png")
plt.show()

# 2. SOURCE PERFORMANCE COMPARISON
print("\n📊 Creating Source Performance Comparison...")
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Source Performance Comparison Analysis', fontsize=16, fontweight='bold')

# Source wins bar chart
sources = list(source_counts.index)
wins = list(source_counts.values)
colors = ['#FFD700' if s == 'optimization-trials' else '#87CEEB' for s in sources]
bars = axes[0,0].bar(sources, wins, color=colors, edgecolor='black')
axes[0,0].set_xlabel('Source')
axes[0,0].set_ylabel('Number of Wins')
axes[0,0].set_title('Source Performance (Task Wins)')
axes[0,0].tick_params(axis='x', rotation=45)
for bar, value in zip(bars, wins):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                   str(value), ha='center', va='bottom', fontweight='bold')

# Source coverage pie chart
coverage_data = {source: len(tasks) for source, tasks in source_task_coverage.items()}
if coverage_data:
    axes[0,1].pie(coverage_data.values(), labels=coverage_data.keys(), autopct='%1.1f%%', 
                  startangle=90, colors=plt.cm.Set3.colors)
    axes[0,1].set_title('Task Coverage by Source')

# Source performance over tasks
axes[1,0].set_xlabel('Task Number')
axes[1,0].set_ylabel('Cumulative Wins')
axes[1,0].set_title('Source Performance Timeline')
for source in source_performance_timeline.keys():
    if source_performance_timeline[source]:
        cumulative_wins = list(range(1, len(source_performance_timeline[source]) + 1))
        axes[1,0].plot(sorted(source_performance_timeline[source]), cumulative_wins, 
                      marker='o' if source == 'optimization-trials' else '', 
                      label=source, alpha=0.8, linewidth=2)
axes[1,0].legend(loc='best')
axes[1,0].grid(True, alpha=0.3)

# Source synergy heatmap preparation
if len(task_source_comparison) > 0:
    # Create a matrix showing how often each source is best for each task range
    task_ranges = [(i*50+1, min((i+1)*50, 400)) for i in range(8)]
    source_names = list(set(stats['source_used']))
    heatmap_data = np.zeros((len(source_names), len(task_ranges)))
    
    for task_idx, source in enumerate(stats['source_used']):
        task_num = stats['task_nums'][task_idx]
        range_idx = (task_num - 1) // 50
        if range_idx < len(task_ranges) and source in source_names:
            source_idx = source_names.index(source)
            heatmap_data[source_idx, range_idx] += 1
    
    im = axes[1,1].imshow(heatmap_data, cmap='YlOrRd', aspect='auto')
    axes[1,1].set_xticks(range(len(task_ranges)))
    axes[1,1].set_xticklabels([f'{r[0]}-{r[1]}' for r in task_ranges], rotation=45)
    axes[1,1].set_yticks(range(len(source_names)))
    axes[1,1].set_yticklabels(source_names)
    axes[1,1].set_xlabel('Task Range')
    axes[1,1].set_ylabel('Source')
    axes[1,1].set_title('Source Dominance by Task Range')
    plt.colorbar(im, ax=axes[1,1])

plt.tight_layout()
plt.savefig('/kaggle/working/source_performance_comparison.png', dpi=150, bbox_inches='tight')
print("✅ Saved: source_performance_comparison.png")
plt.show()

# 3. OPTIMIZATION EFFECTIVENESS
print("\n📊 Creating Optimization Effectiveness Visualization...")
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Optimization Effectiveness Analysis', fontsize=16, fontweight='bold')

# Size reduction scatter
axes[0,0].scatter(stats['original_sizes'], stats['final_sizes'], 
                  alpha=0.5, c=stats['potential_scores'], cmap='viridis', s=30)
axes[0,0].plot([0, max(stats['original_sizes'])], [0, max(stats['original_sizes'])], 
               'r--', alpha=0.5, label='No reduction')
axes[0,0].set_xlabel('Original Size (bytes)')
axes[0,0].set_ylabel('Final Size (bytes)')
axes[0,0].set_title('Size Reduction Scatter')
axes[0,0].legend()

# Minification effectiveness
axes[0,1].hist(stats['minification_reduction'], bins=30, alpha=0.7, 
               color='#8e44ad', edgecolor='black')
axes[0,1].axvline(np.mean(stats['minification_reduction']), color='red', 
                  linestyle='--', linewidth=2, label=f'Mean: {np.mean(stats["minification_reduction"]):.1f}%')
axes[0,1].set_xlabel('Minification Reduction (%)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Minification Effectiveness')
axes[0,1].legend()

# Compression beneficial pie
compression_data = ['Beneficial', 'Not Beneficial']
compression_counts = [compression_beneficial_count, analyzed_tasks - compression_beneficial_count]
colors = ['#2ecc71', '#e74c3c']
axes[0,2].pie(compression_counts, labels=compression_data, colors=colors, 
              autopct='%1.1f%%', startangle=90)
axes[0,2].set_title('Compression Benefit Analysis')

# Efficiency score distribution
axes[1,0].hist(stats['efficiency_score'], bins=30, alpha=0.7, 
               color='#3498db', edgecolor='black')
axes[1,0].set_xlabel('Efficiency Score')
axes[1,0].set_ylabel('Frequency')
axes[1,0].set_title('Task Efficiency Distribution')

# Optimization potential vs actual reduction
actual_reduction = [(orig - final) for orig, final in zip(stats['original_sizes'], stats['final_sizes'])]
axes[1,1].scatter(stats['optimization_potential'], actual_reduction, alpha=0.5, s=20)
axes[1,1].set_xlabel('Optimization Potential')
axes[1,1].set_ylabel('Actual Reduction (bytes)')
axes[1,1].set_title('Optimization Potential vs Actual')
axes[1,1].grid(True, alpha=0.3)

# Task difficulty vs final size
difficulty_order = ['Easy', 'Medium', 'Hard', 'Expert']
difficulty_sizes = {d: [] for d in difficulty_order}
for diff, size in zip(stats['task_difficulty'], stats['final_sizes']):
    difficulty_sizes[diff].append(size)

bp = axes[1,2].boxplot([difficulty_sizes[d] for d in difficulty_order], 
                        labels=difficulty_order, patch_artist=True)
colors = ['#2ecc71', '#f39c12', '#e74c3c', '#9b59b6']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
axes[1,2].set_xlabel('Difficulty')
axes[1,2].set_ylabel('Final Size (bytes)')
axes[1,2].set_title('Size Distribution by Difficulty')

plt.tight_layout()
plt.savefig('/kaggle/working/optimization_effectiveness.png', dpi=150, bbox_inches='tight')
print("✅ Saved: optimization_effectiveness.png")
plt.show()

# 4. CODE PATTERNS AND TECHNIQUES
print("\n📊 Creating Code Patterns Analysis...")
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Code Patterns and Techniques Analysis', fontsize=16, fontweight='bold')

# Top patterns bar chart
top_patterns = sorted(pattern_frequency.items(), key=lambda x: x[1], reverse=True)[:15]
pattern_names = [p[0].replace('_', ' ').title() for p in top_patterns]
pattern_counts = [p[1] for p in top_patterns]

axes[0,0].barh(pattern_names, pattern_counts, color='#16a085')
axes[0,0].set_xlabel('Occurrences')
axes[0,0].set_title('Top 15 Code Patterns')
axes[0,0].invert_yaxis()

# Pattern correlation heatmap
if len(pattern_frequency) > 5:
    # Calculate co-occurrence of patterns
    pattern_list = list(pattern_frequency.keys())[:10]
    cooccurrence = np.zeros((len(pattern_list), len(pattern_list)))
    
    for patterns_dict in stats['code_patterns']:
        for source_patterns in patterns_dict.values():
            for i, p1 in enumerate(pattern_list):
                for j, p2 in enumerate(pattern_list):
                    if source_patterns.get(p1, 0) > 0 and source_patterns.get(p2, 0) > 0:
                        cooccurrence[i, j] += 1
    
    im = axes[0,1].imshow(cooccurrence, cmap='Blues', aspect='auto')
    axes[0,1].set_xticks(range(len(pattern_list)))
    axes[0,1].set_xticklabels([p.replace('_', ' ')[:10] for p in pattern_list], rotation=45)
    axes[0,1].set_yticks(range(len(pattern_list)))
    axes[0,1].set_yticklabels([p.replace('_', ' ')[:10] for p in pattern_list])
    axes[0,1].set_title('Pattern Co-occurrence Matrix')
    plt.colorbar(im, ax=axes[0,1])

# Complexity metrics distribution
complexity_distributions = {
    'Line Count': [m.get('line_count', 0) for m in stats['complexity_metrics']],
    'Function Count': [m.get('function_count', 0) for m in stats['complexity_metrics']],
    'Loop Count': [m.get('loop_count', 0) for m in stats['complexity_metrics']],
    'Lambda Count': [m.get('lambda_count', 0) for m in stats['complexity_metrics']]
}

bp = axes[1,0].boxplot(list(complexity_distributions.values()), 
                        labels=list(complexity_distributions.keys()),
                        patch_artist=True)
for patch, color in zip(bp['boxes'], plt.cm.Set2.colors):
    patch.set_facecolor(color)
axes[1,0].set_ylabel('Count')
axes[1,0].set_title('Code Complexity Metrics')
axes[1,0].tick_params(axis='x', rotation=45)

# Word cloud of patterns
if pattern_frequency:
    pattern_text = ' '.join([f"{pattern} " * count for pattern, count in pattern_frequency.items()])
    if pattern_text.strip():
        wordcloud = WordCloud(width=400, height=300, background_color='white', 
                            colormap='viridis').generate(pattern_text)
        axes[1,1].imshow(wordcloud, interpolation='bilinear')
        axes[1,1].axis('off')
        axes[1,1].set_title('Pattern Word Cloud')

plt.tight_layout()
plt.savefig('/kaggle/working/code_patterns_analysis.png', dpi=150, bbox_inches='tight')
print("✅ Saved: code_patterns_analysis.png")
plt.show()

# 5. TASK PROGRESSION ANALYSIS
print("\n📊 Creating Task Progression Analysis...")
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Task Progression and Trends Analysis', fontsize=16, fontweight='bold')

# Task size progression
if task_size_progression:
    task_nums_prog, sizes_prog = zip(*task_size_progression)
    axes[0,0].plot(task_nums_prog, sizes_prog, alpha=0.5, linewidth=1)
    
    # Add rolling average
    window_size = 20
    if len(sizes_prog) >= window_size:
        rolling_avg = pd.Series(sizes_prog).rolling(window=window_size, center=True).mean()
        axes[0,0].plot(task_nums_prog, rolling_avg, 'r-', linewidth=2, label=f'{window_size}-task rolling avg')
    
    axes[0,0].set_xlabel('Task Number')
    axes[0,0].set_ylabel('Final Size (bytes)')
    axes[0,0].set_title('Task Size Progression')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

# Score progression
axes[0,1].plot(stats['task_nums'], stats['potential_scores'], alpha=0.5, linewidth=1)
if len(stats['potential_scores']) >= 20:
    rolling_scores = pd.Series(stats['potential_scores']).rolling(window=20, center=True).mean()
    axes[0,1].plot(stats['task_nums'], rolling_scores, 'g-', linewidth=2, label='20-task rolling avg')
axes[0,1].set_xlabel('Task Number')
axes[0,1].set_ylabel('Potential Score')
axes[0,1].set_title('Score Progression')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Difficulty distribution over task ranges
task_ranges = [(i*50+1, min((i+1)*50, 400)) for i in range(8)]
difficulty_by_range = {d: [0]*len(task_ranges) for d in difficulty_order}

for task_num, difficulty in zip(stats['task_nums'], stats['task_difficulty']):
    range_idx = (task_num - 1) // 50
    if range_idx < len(task_ranges):
        difficulty_by_range[difficulty][range_idx] += 1

bottom = np.zeros(len(task_ranges))
colors = {'Easy': '#2ecc71', 'Medium': '#f39c12', 'Hard': '#e74c3c', 'Expert': '#9b59b6'}
for difficulty in difficulty_order:
    axes[1,0].bar(range(len(task_ranges)), difficulty_by_range[difficulty], 
                  bottom=bottom, label=difficulty, color=colors[difficulty])
    bottom += difficulty_by_range[difficulty]

axes[1,0].set_xticks(range(len(task_ranges)))
axes[1,0].set_xticklabels([f'{r[0]}-{r[1]}' for r in task_ranges])
axes[1,0].set_xlabel('Task Range')
axes[1,0].set_ylabel('Count')
axes[1,0].set_title('Difficulty Distribution by Task Range')
axes[1,0].legend()

# Compression benefit over tasks
compression_window = []
compression_x = []
compression_y = []

for i, (task_num, beneficial) in enumerate(zip(stats['task_nums'], stats['compression_beneficial'])):
    compression_window.append(1 if beneficial else 0)
    if len(compression_window) > 50:
        compression_window.pop(0)
    if len(compression_window) >= 10:
        compression_x.append(task_num)
        compression_y.append(sum(compression_window) / len(compression_window) * 100)

if compression_x:
    axes[1,1].plot(compression_x, compression_y, linewidth=2, color='#e67e22')
    axes[1,1].fill_between(compression_x, compression_y, alpha=0.3, color='#e67e22')
    axes[1,1].set_xlabel('Task Number')
    axes[1,1].set_ylabel('Compression Benefit Rate (%)')
    axes[1,1].set_title('Compression Effectiveness Over Tasks (50-task window)')
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].axhline(y=50, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('/kaggle/working/task_progression_analysis.png', dpi=150, bbox_inches='tight')
print("✅ Saved: task_progression_analysis.png")
plt.show()

# 6. INTERACTIVE PLOTLY DASHBOARD
print("\n📊 Creating Interactive Dashboard...")
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Create subplots
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Size Distribution', 'Score Distribution', 'Source Wins',
                    'Size Reduction', 'Compression Benefit', 'Task Difficulty',
                    'Pattern Frequency', 'Efficiency Score', 'Task Progression'),
    specs=[[{"type": "histogram"}, {"type": "histogram"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "pie"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "histogram"}, {"type": "scatter"}]]
)

# 1. Size distribution
fig.add_trace(go.Histogram(x=stats['final_sizes'], name='Final Size', 
                           marker_color='lightblue', nbinsx=30),
              row=1, col=1)

# 2. Score distribution
fig.add_trace(go.Histogram(x=stats['potential_scores'], name='Potential Score',
                           marker_color='lightgreen', nbinsx=30),
              row=1, col=2)

# 3. Source wins
fig.add_trace(go.Bar(x=list(source_counts.index), y=list(source_counts.values),
                     name='Source Wins', marker_color='gold'),
              row=1, col=3)

# 4. Size reduction scatter
fig.add_trace(go.Scatter(x=stats['original_sizes'], y=stats['final_sizes'],
                        mode='markers', name='Size Reduction',
                        marker=dict(size=5, color=stats['potential_scores'], 
                                  colorscale='Viridis', showscale=True)),
              row=2, col=1)

# 5. Compression benefit pie
fig.add_trace(go.Pie(labels=['Beneficial', 'Not Beneficial'],
                     values=[compression_beneficial_count, 
                            analyzed_tasks - compression_beneficial_count],
                     name='Compression'),
              row=2, col=2)

# 6. Task difficulty pie
fig.add_trace(go.Pie(labels=list(difficulty_counts.index),
                     values=list(difficulty_counts.values),
                     name='Difficulty'),
              row=2, col=3)

# 7. Top patterns
top_10_patterns = sorted(pattern_frequency.items(), key=lambda x: x[1], reverse=True)[:10]
fig.add_trace(go.Bar(x=[p[1] for p in top_10_patterns],
                     y=[p[0] for p in top_10_patterns],
                     orientation='h', name='Patterns',
                     marker_color='purple'),
              row=3, col=1)

# 8. Efficiency score
fig.add_trace(go.Histogram(x=stats['efficiency_score'], name='Efficiency',
                           marker_color='orange', nbinsx=30),
              row=3, col=2)

# 9. Task progression
fig.add_trace(go.Scatter(x=stats['task_nums'], y=stats['final_sizes'],
                        mode='lines', name='Size Progression',
                        line=dict(color='blue', width=1)),
              row=3, col=3)

# Update layout
fig.update_layout(height=1200, showlegend=False, 
                 title_text="Code Golf Analysis Interactive Dashboard")
fig.update_xaxes(title_text="Size (bytes)", row=1, col=1)
fig.update_xaxes(title_text="Score", row=1, col=2)
fig.update_xaxes(title_text="Original Size", row=2, col=1)
fig.update_yaxes(title_text="Final Size", row=2, col=1)
fig.update_xaxes(title_text="Task Number", row=3, col=3)
fig.update_yaxes(title_text="Size (bytes)", row=3, col=3)

# Save interactive plot
fig.write_html("/kaggle/working/interactive_dashboard.html")
print("✅ Saved: interactive_dashboard.html")
fig.show()

# 7. CORRELATION ANALYSIS
print("\n📊 Creating Correlation Analysis...")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Correlation Analysis', fontsize=16, fontweight='bold')

# Prepare correlation data
corr_data = pd.DataFrame({
    'Original Size': stats['original_sizes'],
    'Final Size': stats['final_sizes'],
    'Score': stats['potential_scores'],
    'Compression Ratio': stats['compression_ratios'],
    'Efficiency': stats['efficiency_score'],
    'Minification %': stats['minification_reduction']
})

# Correlation matrix
correlation_matrix = corr_data.corr()
im = axes[0].imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(correlation_matrix.columns)))
axes[0].set_xticklabels(correlation_matrix.columns, rotation=45)
axes[0].set_yticks(range(len(correlation_matrix.columns)))
axes[0].set_yticklabels(correlation_matrix.columns)
axes[0].set_title('Feature Correlation Matrix')

# Add correlation values
for i in range(len(correlation_matrix)):
    for j in range(len(correlation_matrix)):
        text = axes[0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                           ha="center", va="center", color="black" if abs(correlation_matrix.iloc[i, j]) < 0.5 else "white")

plt.colorbar(im, ax=axes[0])

# Pairplot for key metrics
key_metrics = pd.DataFrame({
    'Size': stats['final_sizes'],
    'Score': stats['potential_scores'],
    'Efficiency': stats['efficiency_score']
})

# Scatter matrix
from pandas.plotting import scatter_matrix
scatter_matrix(key_metrics, ax=axes[1], figsize=(6, 6), diagonal='hist', alpha=0.5)
axes[1].set_title('Key Metrics Relationships')

plt.tight_layout()
plt.savefig('/kaggle/working/correlation_analysis.png', dpi=150, bbox_inches='tight')
print("✅ Saved: correlation_analysis.png")
plt.show()

# 8. SUMMARY INFOGRAPHIC
print("\n📊 Creating Summary Infographic...")
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('Code Golf Analysis Summary Infographic', fontsize=20, fontweight='bold', y=0.98)

# Key metrics boxes
metrics_to_display = [
    ('Tasks Analyzed', f'{analyzed_tasks}/400', '#3498db'),
    ('Total Score', f'{total_potential_score:,}', '#2ecc71'),
    ('Avg Size', f'{np.mean(stats["final_sizes"]):.0f} bytes', '#9b59b6'),
    ('Compression Rate', f'{compression_beneficial_count/analyzed_tasks*100:.1f}%', '#e74c3c'),
    ('Best Source', source_counts.index[0] if len(source_counts) > 0 else 'N/A', '#f39c12'),
    ('Submission Size', f'{submission_size/1024:.1f} KB', '#16a085'),
    ('Analysis Time', f'{analysis_time:.1f}s', '#95a5a6'),
    ('Score Efficiency', f'{total_potential_score/(2500*analyzed_tasks)*100:.1f}%', '#e67e22')
]

for i, (label, value, color) in enumerate(metrics_to_display):
    ax = fig.add_subplot(gs[0, i % 4])
    ax.text(0.5, 0.7, value, ha='center', va='center', fontsize=20, fontweight='bold', color=color)
    ax.text(0.5, 0.3, label, ha='center', va='center', fontsize=12)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=False, edgecolor=color, linewidth=2))

# Remaining visualizations for the infographic
# ... [Add more summary visualizations as needed]

plt.savefig('/kaggle/working/summary_infographic.png', dpi=150, bbox_inches='tight')
print("✅ Saved: summary_infographic.png")
plt.show()

# Final comprehensive summary
print("\n" + "="*100)
print("""
╔═══════════════════════════════════════════════════════════════════════════════════════════════╗
║                           📋 ULTRA-ENHANCED FINAL SUMMARY                                    ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
""")

summary_data = [
    ("ANALYSIS PERFORMANCE", [
        ("Analysis time", f"{analysis_time:.2f} seconds"),
        ("Tasks analyzed", f"{analyzed_tasks}/400"),
        ("Sources compared", str(len(source_counts))),
        ("Total comparisons made", str(tasks_analyzed * len(analysis_sources))),
        ("Code analyzed", f"{total_code_analyzed:,} bytes"),
        ("Analysis speed", f"{total_code_analyzed/analysis_time/1024:.1f} KB/sec"),
    ]),
    ("OPTIMIZATION RESULTS", [
        ("Compression beneficial for", f"{compression_beneficial_count} tasks ({compression_beneficial_count/analyzed_tasks*100:.1f}%)"),
        ("Total bytes saved", f"{total_bytes_saved:,} bytes"),
        ("Average minification", f"{np.mean(stats['minification_reduction']):.1f}%"),
        ("High optimization potential", f"{len(high_potential_tasks)} tasks"),
    ]),
    ("SCORING METRICS", [
        ("Potential score", f"{total_potential_score:,}"),
        ("Average per task", f"{total_potential_score/analyzed_tasks:.1f}"),
        ("Perfect scores (≥2450)", f"{sum(1 for s in stats['potential_scores'] if s >= 2450)} tasks"),
        ("Score efficiency", f"{total_potential_score/(2500*analyzed_tasks)*100:.1f}%"),
    ]),
    ("OPTIMIZATION TRIALS SUBMISSION", [
        ("Source used", "golf-code-optimization-trials"),
        ("Location", "/kaggle/working/submission.zip"),
        ("Submission size", f"{submission_size:,} bytes ({submission_size/1024:.1f} KB)" if submission_size > 0 else "Not found"),
        ("Size per task", f"{submission_size/400:.1f} bytes average" if submission_size > 0 else "N/A"),
    ]),
    ("VISUALIZATIONS CREATED", [
        ("Size Distribution Analysis", "size_distribution_analysis.png"),
        ("Source Performance", "source_performance_comparison.png"),
        ("Optimization Effectiveness", "optimization_effectiveness.png"),
        ("Code Patterns", "code_patterns_analysis.png"),
        ("Task Progression", "task_progression_analysis.png"),
        ("Interactive Dashboard", "interactive_dashboard.html"),
        ("Correlation Analysis", "correlation_analysis.png"),
        ("Summary Infographic", "summary_infographic.png"),
    ]),
]

for section_title, metrics in summary_data:
    print(f"\n{'─'*100}")
    print(f"  {section_title}")
    print(f"{'─'*100}")
    for metric, value in metrics:
        print(f"  {metric:<40} {value:<40}")

print("\n" + "="*100)
print("\n✨ Ultra-enhanced analysis complete with multiple visualizations!")
print("🚀 Optimization trials submission ready at /kaggle/working/submission.zip")
print("📊 Check generated files for comprehensive visualizations:")
print("   • 8 detailed PNG visualizations")
print("   • 1 interactive HTML dashboard")

print("\n🎯 Key Insights:")
print("   • The golf-code-optimization-trials solution is being used for submission")
print(f"   • Best performing source in analysis: {source_counts.index[0] if len(source_counts) > 0 else 'N/A'}")
print(f"   • Compression beneficial for {compression_beneficial_count/analyzed_tasks*100:.1f}% of tasks")
print(f"   • Total optimization potential: {sum(stats['optimization_potential']):.1f}")

print("\n" + "="*100)
print("📈 ANALYSIS COMPLETE - OPTIMIZATION TRIALS SUBMISSION READY FOR COMPETITION")
print("="*100)